In [183]:
import pandas as pd
import numpy as np
from datetime import datetime


In [184]:

#CARGAR EL ARCHIVO CON LOS DATOS DE LA AGENCIA 14
ruta_agencia14 = r"C:\Users\sgarcial\Documents\ETB\Agencia 1\SABANA TEMPORAL\DatosAgencia_Enero.xlsx"
df = pd.read_excel(
    ruta_agencia14,
    usecols=['Telefono', 'Nivel_1', 'Franja','NombreServicio', 'Fuente', 'Venta',  'Nivel3'],
    sheet_name='Datos'
)

#CARGAR ARCHIVO CRITERIOS DE LA AGENCIA 14
ruta_criterios = r"C:\Users\sgarcial\Documents\ETB\Agencia 1\SABANA TEMPORAL\Analisis reinyecciones 960014 Agencia Dg_V2 (1).xlsx"
df_criterios = pd.read_excel(
    ruta_criterios,
    sheet_name="Criterios",
    header=None,
)

df_criterios_telefono = df_criterios.copy()

In [185]:
#APLICAR FILTROS A LAS COLUMNAS NIVEL1, NOMBRE SERVICIO Y VENTA

#FILTRO A LA COLUMNA NIVEL 1
valores_excluir_Nivel1 = [
    "ATENCIÓN", "ATENCION",
    "ATENCIÓN, CONTACTO NO UTIL", "ATENCION, CONTACTO NO UTIL",
    "ATENCIÓN, EN PROCESO", "ATENCION, EN PROCESO",
    "ATENCIÓN, NO VENTA", "ATENCION, NO VENTA",
    "ATENCION, SERVICIO AL CLIENTE", "ATENCION, SERVICIO AL CLIENTE",
    "ATENCIÓN, VENTA HOGARES", "ATENCION, VENTA HOGARES",
    "SERVICIO AL CLIENTE",
    "VENTA",
    "VENTA FTTH + MÓVIL", "VENTA FTTH + MOVIL"
    "VENTA HOGARES",
    "VENTA MOVIL", "VENTA MÓVIL",
    "VENTA NEGOCIO",
    "VENTA NEGOCIOS"]
df["Nivel_1"] = df["Nivel_1"].astype(str).str.strip().str.upper()
df = df[~df["Nivel_1"].isin(valores_excluir_Nivel1)]

#FILTRO A LA COLUMNA NOMBRE SERVICIO
valores_excluir_servicio = [
    "AGENCIA ETB PD REINYECCION", "AGENCIA ETB PD REINYECCIÓN",
    "AGENCIA ETB PT",
    "AGENCIA ETB PU",
    "BBDD_IR",
    "REFERIDOS_C2C_2"]
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df = df[~df["NombreServicio"].isin(valores_excluir_servicio)]

#FILTRO A LA COLUMNA VENTA
df["Venta"] = pd.to_numeric(df["Venta"], errors="coerce")
df = df[df["Venta"] == 0]


In [186]:
# Supón que tu df ya existe y la columna a transformar se llama 'col'
diccionario_franjas = {
    '9:00':  [0, 30, 100, 130, 200, 230, 300, 330, 400, 430, 500, 530, 600, 630, 700, 730, 800, 830],
    '12:00': [900, 930, 1000, 1030, 1100, 1130, 1200, 1230],
    '16:00': [1300, 1330, 1400, 1430, 1500, 1530, 1600, 1630],
    '20:00': [1700, 1730, 1800, 1830, 1900, 1930, 2000, 2030, 2100, 2130, 2200, 2230, 2300, 2330],
}

inv = {v: k for k, vals in diccionario_franjas.items() for v in vals}

# Supón que tu df ya existe y la columna a transformar se llama 'col'
df['Franja'] = df['Franja'].map(inv)

In [187]:
#ELIMANR GUIONES Y ESPACIOS DE LA COLUMNA Telefono
df["Telefono"] = (
    df["Telefono"]
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace("-", "", regex=False)
)


#CREAR COLUMNA TELEFONO2
df["TELEFONO2"] = (
    df["Telefono"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)
    .str[-10:]
)

df = df.drop(columns=["Telefono"])

In [188]:
#CREAR COLUMNA CATEGORIA, ANTERIORMENTE =BUSCARV(Nombre Servicio; &#39; Análisis de reinyección Hoja1&#39;!A:B; 2; 0)
df_criterios_categoria = df_criterios.copy()
df_criterios_categoria = df_criterios_categoria.iloc[:, [0, 1]]
df_criterios_categoria.columns = ["NombreServicio", "CATEGORIA"]
#df_criterios_categoria = df_criterios_categoria.dropna(subset=["NombreServicio"])
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df_criterios_categoria["NombreServicio"] = df_criterios_categoria["NombreServicio"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios_categoria,
    on="NombreServicio",
    how="left"
)

df2 = df.copy()

In [189]:
#CREAR COLUMNA FUENTE, ANTERIORMENTE =BUSCARV(Fuente; &#39; Análisis de reinyección Hoja3&#39;!G:K; 5; 0)

# CARGAR LA HOJA CON EL RANGO
df_fuente = pd.read_excel(
    ruta_criterios,
    sheet_name="ANALISIS EFECTIVIDAD",
    header=None,
    engine="openpyxl"
)

df_criterios_fuente_telefono = df_fuente.copy()


# G = 6, K = 10 → 6:11
df_fuente = df_fuente.iloc[30:155, 6:11]

df_fuente.columns = [
    "Fuente",     # Columna G → CLAVE
    "COL_H",
    "COL_I",
    "COL_J",
    "FUENTE"      # Columna K → RESULTADO BUSCARV
]


df["Fuente"] = df["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()


df = df.merge(
    df_fuente[["Fuente", "FUENTE"]],
    on="Fuente",
    how="left"
)

#df['FUENTE'] = df['FUENTE'].fillna('BAJA')

In [190]:
#CREAR COLUMNA TIPIFICACION, ANTERIORMENTE =BUSCARV(Nivel_3; &#39;Análisis de reinyección Hoja1&#39;!D:F; 2; 0)
# D = 3, E = 5 → 3:5
df_criterios = df_criterios.iloc[:, 3:5]

df_criterios.columns = [
    "Nivel3",        # Columna D → CLAVE
    "TIPIFICACION"]  # Columna E → RESULTADO BUSCARV
    

#df_criterios = df_criterios.dropna(subset=["Nivel3"])
df["Nivel3"] = df["Nivel3"].astype(str).str.strip().str.upper()
df_criterios["Nivel3"] = df_criterios["Nivel3"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)


In [191]:
#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "1evd-Dbx3a1o9vONycdM3lcIHcDHtCFk0"
sheet_name = "CONSOLIDADO%20WPP"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"


# 1) Leer Google Sheets (✓ correcto)
df_CONSOLIDADO_WPP_agencia1 = pd.read_csv(
    url,
    usecols=["21"],
    dtype=str
)

df_CONSOLIDADO_WPP_agencia1 = df_CONSOLIDADO_WPP_agencia1.rename(
    columns={df_CONSOLIDADO_WPP_agencia1.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)



# 2) Normalizar (agregar esta parte que faltaba)
df["TELEFONO2"] = df["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()
)

df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"]
    .astype(str)
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace("-", "", regex=False)
)

# 2️⃣ Eliminar valores nulos, cadenas vacías o 'nan' literales
df_CONSOLIDADO_WPP_agencia1 = df_CONSOLIDADO_WPP_agencia1[
    df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"].notna() &
    (df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"] != "") &
    (df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"].str.lower() != "nan")
]

# 3️⃣ (Opcional) Mantener solo filas con números reales
df_CONSOLIDADO_WPP_agencia1 = df_CONSOLIDADO_WPP_agencia1[
    df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"].str.fullmatch(r"\d+")
]

# 3) Merge CORREGIDO (lookup como BUSCARV)
df = df.merge(
    df_CONSOLIDADO_WPP_agencia1.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS con los teléfonos encontrados
df["YAREINYECTADOS"] = df["TELEFONOS_GOOGLE_SHEETS"]
df.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)




In [192]:
print(df_CONSOLIDADO_WPP_agencia1)

    TELEFONOS_GOOGLE_SHEETS
0                3006108867
1                3057125239
2                3115717739
3                3133146282
4                3054754885
..                      ...
573              3108875831
574              3125422858
575              3125480960
576              3044374868
577              3114858418

[578 rows x 1 columns]


In [210]:
df_chat_14 = df.copy()
df_chat_14 = df_chat_14[df_chat_14['CATEGORIA'] == 'CHAT']  # ← CRÍTICO
df_chat_14['FUENTE'] = df_chat_14['FUENTE'].fillna('BAJA')
df_chat_14 = df_chat_14[df_chat_14['TIPIFICACION'] == 'TENER EN CUENTA']
df_chat_14 = df_chat_14[~df_chat_14['YAREINYECTADOS'].apply(pd.to_numeric, errors='coerce').notna()]

df_chat_14 = df_chat_14.drop_duplicates(subset=["TELEFONO2", "CATEGORIA"], keep="first")
df_chat_14.rename(columns={'Franja': 'FRANJA'}, inplace=True)

df_chat_14 = df_chat_14[['TELEFONO2', 'FUENTE', 'FRANJA']].sort_values(by='TELEFONO2')
df_chat_14["TELEFONO2"] = pd.to_numeric(df_chat_14["TELEFONO2"], errors="coerce")
df_chat_14 = df_chat_14[(df_chat_14["TELEFONO2"] >= 3000000000) & (df_chat_14["TELEFONO2"] <= 3999999999)]


df_chat_14.rename(columns={'TELEFONO2': 'TELEFONO'}, inplace=True)

In [213]:
print(df_chat_14.shape)
df_chat_14.tail(10)

(599, 3)


,TELEFONO,FUENTE,FRANJA
6628,3503775210,BAJA,12:00
4966,3504268681,BAJA,16:00
8886,3505087835,ALTA,20:00
7632,3506026152,ALTA,12:00
5322,3506221342,ALTA,20:00
8520,3506542184,BAJA,16:00
8075,3507239483,ALTA,16:00
6624,3507815939,BAJA,12:00
7304,3508102874,ALTA,12:00
6708,3508102878,BAJA,20:00


In [214]:
df_chat_14 = df_chat_14.drop_duplicates()
print(df_chat_14.shape)

(599, 3)


In [196]:
df_chat_14['CANAL'] = None

for franja, idx in df_chat_14.groupby('FRANJA').groups.items():
    # idx YA son índices válidos del df
    n = len(idx)
    n_inbox = int(n * 0.7)
    canales = np.array(['INBOX'] * n_inbox + ['CHAT'] * (n - n_inbox))
    np.random.shuffle(canales)
    df_chat_14.loc[idx, 'CANAL'] = canales

In [197]:
# Verificar conteo y proporción por Franja
verificacion = (
    df_chat_14
    .groupby(['FRANJA', 'CANAL'])
    .size()
    .unstack(fill_value=0)
)

# Agregar columnas de totales y porcentajes
verificacion['TOTAL'] = verificacion.sum(axis=1)
verificacion['%INBOX'] = (verificacion['INBOX'] / verificacion['TOTAL'] * 100).round(1)
verificacion['%CHAT'] = (verificacion['CHAT'] / verificacion['TOTAL'] * 100).round(1)

print(verificacion)


CANAL   CHAT  INBOX  TOTAL  %INBOX  %CHAT
FRANJA                                   
12:00     47    108    155    69.7   30.3
16:00     45    104    149    69.8   30.2
20:00     70    162    232    69.8   30.2
9:00      19     44     63    69.8   30.2


In [198]:
# Suponiendo que tu df principal se llama df_chat_14
# y tiene las columnas 'Franja' y 'CANAL'

df_900_inbox  = df_chat_14[(df_chat_14['FRANJA'] == '9:00')  & (df_chat_14['CANAL'] == 'INBOX')]
df_900_chat   = df_chat_14[(df_chat_14['FRANJA'] == '9:00')  & (df_chat_14['CANAL'] == 'CHAT')]

df_1200_inbox = df_chat_14[(df_chat_14['FRANJA'] == '12:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_1200_chat  = df_chat_14[(df_chat_14['FRANJA'] == '12:00') & (df_chat_14['CANAL'] == 'CHAT')]

df_1600_inbox = df_chat_14[(df_chat_14['FRANJA'] == '16:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_1600_chat  = df_chat_14[(df_chat_14['FRANJA'] == '16:00') & (df_chat_14['CANAL'] == 'CHAT')]

df_2000_inbox = df_chat_14[(df_chat_14['FRANJA'] == '20:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_2000_chat  = df_chat_14[(df_chat_14['FRANJA'] == '20:00') & (df_chat_14['CANAL'] == 'CHAT')]


In [199]:
# Inicializar columna
df_chat_14['CANAL'] = None

# Semilla (opcional, para reproducibilidad)
np.random.seed(42)

# Reparto por franja
for franja, idx in df_chat_14.groupby('FRANJA').groups.items():
    n = len(idx)
    n_inbox = round(n * 0.7)  # usa redondeo para mayor precisión
    n_chat = n - n_inbox

    # Crear el conjunto de etiquetas
    canales = np.array(['INBOX'] * n_inbox + ['CHAT'] * n_chat)
    np.random.shuffle(canales)

    # Asignar al subconjunto correspondiente
    df_chat_14.loc[idx, 'CANAL'] = canales

In [200]:
df2.head()

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA
0,12:00,NO VENTA,C2C_2,003GenericoInternetEx,0,REQUIERE ESCALAMIENTO,3134987378,LLAMADA
1,12:00,ILOCALIZABLES,C2C_2,0021ETBBogotaEx,0,CONTESTADOR,3007297762,LLAMADA
2,12:00,EN PROCESO,C2C_2,0023ETBInternetFr,0,Incompleto,3115613898,LLAMADA
3,16:00,ILOCALIZABLES,C2C_2,0021ETBBogotaEx,0,CONTESTADOR,3228223315,LLAMADA
4,16:00,EN PROCESO,C2C_2,003GenericoInternetEx,0,Llamada cortada,3150687114,LLAMADA


In [201]:
#CREAR LAS COLUMNAS PARA CATEGORIA == TELEFONO

# 1) CREAR COLUMNA FUENTE CON LOS RANGOS DE TELEFONO
df_fuente = df_criterios_fuente_telefono.iloc[:121, 11:16]  # L:P, filas 32-152 ✓
df_fuente.columns = ["Fuente", "COL_M", "COL_N", "COL_O", "FUENTE"]  # P = FUENTE ✓
df2["Fuente"] = df2["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()
df2 = df2.merge(df_fuente[["Fuente", "FUENTE"]], on="Fuente", how="left")


In [202]:
# CREAR COLUMNA TIPIFICACION CON EL RANGO DE TELEFONOS

df_criterios_telefono = df_criterios_telefono.iloc[:125, 7:10]

df_criterios_telefono.columns = [
    "Nivel3",        # Columna H → CLAVE
    "TIPIFICACION",  # Columna I → RESULTADO BUSCARV (2ª columna)
    "COL_J"
]

# Normalización para df2 (TELEFONO)
df2["Nivel3"] = df2["Nivel3"].astype(str).str.strip().str.upper()
df_criterios_telefono["Nivel3"] = df_criterios_telefono["Nivel3"].astype(str).str.strip().str.upper()

# Merge para df2
df2 = df2.merge(
    df_criterios_telefono[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)

In [203]:


#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "1evd-Dbx3a1o9vONycdM3lcIHcDHtCFk0"
sheet_name_voz = "CONSOLIDADO_VOZ"

# NUEVA URL para hoja VOZ
url_voz = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_voz}"

# 1) Leer Google Sheets (igual que el que funciona)
df_CONSOLIDADO_VOZ_agencia1 = pd.read_csv(
    url_voz,
    #usecols=[0],  # ← Columna A (igual que "21" pero índice 0)
    dtype=str
)
df_CONSOLIDADO_VOZ_agencia1 = df_CONSOLIDADO_VOZ_agencia1.rename(
    columns={df_CONSOLIDADO_VOZ_agencia1.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)

# 2) Normalizar para df2
df2["TELEFONO2"] = df2["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_VOZ_agencia1["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_VOZ_agencia1["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()
)

# 3) Merge para df2
df2 = df2.merge(
    df_CONSOLIDADO_VOZ_agencia1.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS para df2
df2["YAREINYECTADOS"] = df2["TELEFONOS_GOOGLE_SHEETS"]
df2.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)

In [204]:

# APLICAR FILTROS
df_llamada_agencia_1 = df2.copy()

df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['CATEGORIA'] == 'LLAMADA']
df_llamada_agencia_1['FUENTE'] = df_llamada_agencia_1['FUENTE'].fillna('BAJA')
df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['TIPIFICACION'] == 'OK']

df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['YAREINYECTADOS'].isna()]
df_llamada_agencia_1 = df_llamada_agencia_1[['TELEFONO2', 'FUENTE',]]

df_llamada_agencia_1 = df_llamada_agencia_1.drop_duplicates(subset=["TELEFONO2"], keep="first")

df_llamada_agencia_1["TELEFONO2"] = pd.to_numeric(df_llamada_agencia_1["TELEFONO2"], errors="coerce")
df_llamada_agencia_1 = df_llamada_agencia_1[(df_llamada_agencia_1["TELEFONO2"] >= 3000000000) & (df_llamada_agencia_1["TELEFONO2"] <= 3999999999)]


df_llamada_agencia_1 = df_llamada_agencia_1.sort_values(by='TELEFONO2')
df_llamada_agencia_1.rename(columns={'TELEFONO2': 'TELEFONO'}, inplace=True)


In [205]:
print(df_llamada_agencia_1.shape)
df_llamada_agencia_1.head(10)

(219, 2)


,TELEFONO,FUENTE
5438,3.002130e+09,BAJA
1398,3.002177e+09,ALTA
5022,3.002409e+09,ALTA
4966,3.002712e+09,BAJA
3729,3.005673e+09,ALTA
4988,3.006606e+09,ALTA
4809,3.006980e+09,ALTA
876,3.007192e+09,ALTA
441,3.007372e+09,ALTA
4974,3.007418e+09,ALTA


In [206]:
# Fecha: dd_mm_yyyy
fecha = datetime.now().strftime('%d_%m_%Y')

# CAMBIAR 0.1 → 01 (sin punto)
nombre_archivo = f"automatizado_0.1_{fecha}_VOZ_AGENCIA_1.xlsx"

# Ruta
ruta_completa = f"C:/Users/sgarcial/Documents/ETB/Agencia 1/LLAMADA/{nombre_archivo}"

# Guardar
df_llamada_agencia_1.to_excel(ruta_completa, index=False)

print(f"✅ Guardado: {nombre_archivo}")

✅ Guardado: automatizado_0.1_08_01_2026_VOZ_AGENCIA_1.xlsx
